<a href="https://colab.research.google.com/github/Anchayadav/AutoTS/blob/main/Recession_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.templates.default ='plotly_white'
import plotly.graph_objects as go

data=pd.read_csv("UK_monthly_gdp.csv")
print(data.head())

  Time Period  GDP Growth
0    /01/2020         0.3
1    /02/2020        -0.5
2    /03/2020        -7.0
3    /04/2020       -20.9
4    /05/2020         3.2


In [ ]:
fig=go.Figure(data=go.Heatmap(z=[data['GDP Growth']],
                              x=data.index,
                             # y=['GDP Growth'],
                              colorscale='Viridis' ))

fig.show()

In [ ]:
data['Time Period']= pd.to_datetime(data['Time Period'],format='/%m/%Y')
data.set_index('Time Period',inplace= True)
quantile_data = data.resample('Q').mean()
print(quantile_data.head())

             GDP Growth
Time Period            
2020-03-31    -2.400000
2020-06-30    -2.900000
2020-09-30     3.500000
2020-12-31     0.200000
2021-03-31     0.033333


In [ ]:
quantile_data['Reccesion'] =((quantile_data['GDP Growth'] < 0 ) & (quantile_data['GDP Growth'].shift(1) < 0 ))
quantile_data['Reccesion'].fillna(False,inplace=True)
fig =go.Figure()
fig.add_trace(go.Scatter(x=quantile_data.index,
                         y=quantile_data['GDP Growth'],
                         name='GDP Growth',
                         line=dict(color='green',width=2)))
fig.add_trace(go.Scatter(x=quantile_data[quantile_data['Reccesion']].index,
                         y=quantile_data[quantile_data['Reccesion']]['GDP Growth'],
                          name='Reccession',line=dict(color='red',width=2)))

fig.update_layout(title='GDP Growth and Recession over time',
                  xaxis_title='Time Period',
                  yaxis_title='GDP Growth')
fig.show()

In [ ]:
quantile_data['Reccesion start'] = quantile_data['Reccesion'].ne(quantile_data['Reccesion'].shift()).cumsum()
recession_periods = quantile_data.groupby('Reccesion start')
recession_duration = recession_periods.size()
recession_severity=recession_periods['GDP Growth'].sum()

fig=go.Figure()
fig.add_trace(go.Bar(x=recession_duration.index,y=recession_duration,
                     name='Recession Duration'))
fig.add_trace(go.Bar(x=recession_severity.index,y=recession_severity,
                     name='recession Seversity'))
fig.update_layout(title='Duration and Severity of Recesion',
                  xaxis_title='Recession peroids',
                  yaxis_title='Duration/Seversity')
fig.show()